In [64]:
import os
import sys
from itertools import permutations # important for getting all the possible k itemsets
import numpy as np
import pandas as pd
# List of all the datasets that are needed
os.listdir(os.getcwd())

['amazon_transactions.csv',
 '.ipynb_checkpoints',
 'MidtermAlgorithm.ipynb',
 'best_buy_items.csv',
 'amazon_items.csv',
 'best_buy_transactions.csv',
 'k-mart_items.csv',
 'k-mart_transactions.csv',
 'nike_items.csv',
 '.git',
 'nike_transactions.csv',
 'ace_hardware_items.csv',
 'ace_hardware_transactions.csv']

In [65]:
def item_k_possibilities(item_names, k):
    '''
    Generate all possible itemsets wit
    '''
    item_k_arrange = permutations(item_names, k)
    possibilities_of_k_items = [item for item in item_k_arrange]
    format_possibilities = [", ".join(x) for x in possibilities_of_k_items]
    return format_possibilities



In [67]:
# Loads a dictionary of datasets that you can select by number
selected_stores = {1: "amazon", 2: "best_buy", 3: "k-mart", 4: "nike", 5: "ace_hardware"}
selected_id = int(input("Enter the store number for the dataset that you want:\n1. Amazon\n2. Best Buy\n3. K-mart\n4. Nike\n5. Ace Hardware\n"))
if selected_id not in selected_stores.keys():
    print("invalid number, There are only 5 choices!Try again next time")
    sys.exit()
item_names = pd.read_csv(f"{os.getcwd()}/{selected_stores[selected_id]}_items.csv")
transactions = pd.read_csv(f"{os.getcwd()}/{selected_stores[selected_id]}_transactions.csv")
print(f"You have selected the {selected_stores[selected_id]} dataset")
# Enter the minimum support and the minimum confidence 
min_support = float(input("Please enter the minimum support percent that you want (1 to 100):\n"))
min_support /= 100
min_confidence = float(input("Please enter the minimum confidence percent that you want (1 to 100):\n"))
min_confidence /= 100


itemset_k1 = item_names.set_index("Item Name").to_dict()["Item #"]

# Split the string by comma to seperate each string in a row
item_k1_names = [name for name in item_names["Item Name"]]


item_k2_names = item_k_possibilities(item_k1_names, 2)
item_k3_names = item_k_possibilities(item_k1_names, 3)
itemset_k2 = pd.DataFrame(item_k2_names)
print(itemset_k2)
itemset_k3 = {}

item_k1_count = transactions['Transaction'].str.split(", ").explode().value_counts()
item_k2 = transactions['Transaction'].str.split(", ")


item_k1 = item_k1_count.to_dict()
# Get the support value for each itemset-1
for k, _ in itemset_k1.items():
    if k not in item_k1:
        itemset_k1[k] = float(0)
    else:
        itemset_k1[k] = float(item_k1[k]) / len(transactions["Transaction"])
        
print(item_k1_count)
print(itemset_k1)


Enter the store number for the dataset that you want:
1. Amazon
2. Best Buy
3. K-mart
4. Nike
5. Ace Hardware
 1


You have selected the amazon dataset


Please enter the minimum support percent that you want (1 to 100):
 22
Please enter the minimum confidence percent that you want (1 to 100):
 33


                                                    0
0    A Beginner’s Guide, Java: The Complete Reference
1                A Beginner’s Guide, Java For Dummies
2   A Beginner’s Guide, Android Programming: The B...
3     A Beginner’s Guide, Head First Java 2nd Edition
4   A Beginner’s Guide, Beginning Programming with...
..                                                ...
85  HTML and CSS: Design and Build Websites, Head ...
86  HTML and CSS: Design and Build Websites, Begin...
87  HTML and CSS: Design and Build Websites, Java ...
88  HTML and CSS: Design and Build Websites, C++ P...
89  HTML and CSS: Design and Build Websites, Effec...

[90 rows x 1 columns]
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
Transaction
Java For Dummies                           13
Android Programming: The Big Nerd Ranch    13
A Beginner’s Guide                         11
Java: The Complete Reference               10
Head First Java 2nd Edition                 8
Begin